# 麥克連大盤脈動_part_2_融資融券餘額  

In [1]:
import os
import sys
from pathlib import Path
import pandas as pd
import duckdb

In [2]:
from finlab import data

In [ ]:
# 引用自建公用模組
sys.path.insert(0, str(Path.cwd().parent))
from proj_util_pkg.settings import ProjEnvSettings

from proj_util_pkg.finlab_api import finlab_manager as flm

## 公用參數設定

In [4]:
pd.set_option("display.max_columns", None)

In [ ]:
# finlab api 服務初始化
finlab = flm.FinlabManager()
data.force_cloud_download = False

In [6]:
# 設定資料庫路徑
TWSTOCK_DATA_ROOT = os.environ.get("hist_data_path")
twstock_db_path = f"{TWSTOCK_DATA_ROOT}/twstock.duckdb"

In [7]:
# TWSTOCK_DATA_ROOT = os.environ.get("hist_data_path")
# twstock_db_path = f"{os.getcwd()}\\data\\twstock.duckdb"

In [8]:
# 連線資料庫
conn_duckdb = duckdb.connect(twstock_db_path)

In [ ]:
# ipython 載入 sql 擴充, 並設置 duckdb 連接
%load_ext sql

%sql conn_duckdb

In [ ]:
%%sql
-- 方法1：查詢當前數據庫中的所有表
SHOW TABLES;

## 融資融券餘額

In [ ]:
# 讀取融資餘額
margin_transactions_info = data.get('margin_balance:融資券總餘額', save_to_storage=True)
margin_transactions_info

In [ ]:
margin_transactions_info.reset_index(inplace=True)
margin_transactions_info = margin_transactions_info[["date", "上市融券交易張數", "上市融資交易金額", "上櫃融券交易張數", "上櫃融資交易金額"]]

# 交易金額換算成億元
margin_transactions_info['上市融資交易金額'] = margin_transactions_info['上市融資交易金額'] / 100000000
margin_transactions_info['上櫃融資交易金額'] = margin_transactions_info['上櫃融資交易金額'] / 100000000

margin_transactions_info

In [ ]:
# 計算融券資減幅度
margin_transactions_info['上市融資增減幅度'] = margin_transactions_info['上市融資交易金額'].diff() / margin_transactions_info['上市融資交易金額'].shift(1) * 100
margin_transactions_info['上櫃融資增減幅度'] = margin_transactions_info['上櫃融資交易金額'].diff() / margin_transactions_info['上櫃融資交易金額'].shift(1) * 100

# 計算融券增減幅度
margin_transactions_info['上市融券增減幅度'] = margin_transactions_info['上市融券交易張數'].diff() / margin_transactions_info['上市融券交易張數'].shift(1) * 100
margin_transactions_info['上櫃融券增減幅度'] = margin_transactions_info['上櫃融券交易張數'].diff() / margin_transactions_info['上櫃融券交易張數'].shift(1) * 100

margin_transactions_info

In [14]:
mt_df = margin_transactions_info[
    [
        'date', '上市融資交易金額', '上市融資增減幅度', '上市融券交易張數', '上市融券增減幅度', 
        '上櫃融資交易金額', '上櫃融資增減幅度', '上櫃融券交易張數', '上櫃融券增減幅度'
    ]
]

In [ ]:
%%sql

-- 檢查表是否存在
CREATE TABLE IF NOT EXISTS tw_sotck_barometer_part_02 (
    date DATE PRIMARY KEY,
    上市融資交易金額 DOUBLE,
    上市融資增減幅度 DOUBLE,
    上市融券交易張數 DOUBLE,
    上市融券增減幅度 DOUBLE,
    上櫃融資交易金額 DOUBLE,
    上櫃融資增減幅度 DOUBLE,
    上櫃融券交易張數 DOUBLE,
    上櫃融券增減幅度 DOUBLE
);

-- 使用UPSERT功能將資料插入或更新
INSERT OR REPLACE INTO tw_sotck_barometer_part_02
SELECT * FROM mt_df;

In [ ]:
%%sql

-- 顯示資料筆數
SELECT COUNT(*) AS total_records FROM tw_sotck_barometer_part_02;

-- 顯示最新的幾筆資料
SELECT * FROM tw_sotck_barometer_part_02 ORDER BY date DESC LIMIT 5;

In [17]:
# 關閉資料庫連線
conn_duckdb.close()
